___
# Titanic: Tomando decisão com Teorema de Bayes

## Atividade - Classificador Naive-Bayes
___

## Índice

- [Titanic](#titanic)
- [Objetivo](#obj)
- [Classificador sem considerar qualquer característica do passageiro](#apriori)
- [Classificador Naive-Bayes com a variável `Sex`](#naivebayes)
- [Classificando os passageiros](#class)
- [Análise de sentimento](#sentimento)
- [Para casa: Estudo Complementar](#casa)
- [Classificador Naive-Bayes com a variável `Pclass`](#classPclass)
- [Separando conjunto de dados em Treinamento (train)  e Teste (test)](#testtrain)
___

<div id="titanic"></div>

## Titanic

Entre as muitas competições disponíveis na plataforma Kaggle, encontra-se a de *Machine Learning* do Titanic conforme apresentado neste [link](https://www.kaggle.com/c/titanic/data).

A base de dados apresentada no arquivo `titanic.xlsx` contém informação de 891 passageiros para os quais foram mensurados as seguintes informações (as categorias serão mantidas em inglês assim como os nomes das variáveis):
 
 * `Survived`: indica se passageiro sobreviveu ou não.
    - 0 = No,
    - 1 = Yes.
 * `Pclass`: indica a classe da passagem (*ticket*).
    - 1 = 1st, 
    - 2 = 2nd, 
    - 3 = 3rd.
 * `Sex`: sexo do passageiro. 
    - female,
    - male.
 * `Age`: idade em anos.
 * `Sibsp`: número de irmãos e/ou cônjuges a bordo do Titanic.
 * `Parch`: número de pais e/ou filhos a bordo do Titanic.
 * `Ticket`: número do *ticket*.
 * `Fare`: tarifa do passageiro.
 * `Cabin`: número da cabine.
 * `Embarked`: portão de embarque.
    - C = Cherbourg, 
    - Q = Queenstown, 
    - S = Southampton.


**Notas:**
A variável `Pclass` pode ser considerada uma *proxy* da situação socioeconômico do passageiro, ou seja, 
1st = Upper, 2nd = Middle,  3rd = Lower.


___

<div id="obj"></div>

## Objetivo

Imagine que você com as suas características estivesse no Titanic no dia no naufrágio. Será que você teria sido um sobrevivente ou não??

Assim, o objetivo dessa atividade é classificar um "novo" passageiro do Titanic como Sobrevivente ou não Sobrevivente levando em considerando (ou seja, aprendendo) o perfil dos sobreviventes e não sobreviventes que de fato estavam no Titanic.

___
Carregando algumas bibliotecas:

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Carregando os dados do arquivo `titanic.xlsx`:

In [ ]:
dados = pd.read_excel('titanic.xlsx')

Para saber quantas linhas (passageiros) e colunas (variáveis) há no *DataFrame* nomeado `dados`:

In [ ]:
dados.shape

___
**Definindo as variáveis qualitativas como categóricas:**

O pandas não tem como saber quais variáveis são qualitativas ao carregar os dados. Então precisamos modificar as colunas explicitamente.

Já deixamos isso pronto para você nesta avaliação:

In [ ]:
dados.loc[:,'Survived'] = dados['Survived'].astype('category')
dados.loc[:,'Pclass'] = dados['Pclass'].astype('category')
dados.loc[:,'Sex'] = dados['Sex'].astype('category')
dados.loc[:,'Cabin'] = dados['Cabin'].astype('category')
dados.loc[:,'Embarked'] = dados['Embarked'].astype('category')

Agora vamos renomear os rótulos das variáveis qualitativas `Survived` e `Pclass` com comando `.cat.categories`:

In [ ]:
dados.Survived.cat.categories = ['No', 'Yes']
dados.Sex.cat.categories = ['female','male']
dados.Pclass.cat.categories = ['1st','2nd','3rd']


___
***DataFrame* `dados` que deve considerar nos passos a seguir:**

In [ ]:
dados.head(3)

<div id="apriori"></div>

___
## Classificador sem considerar qualquer característica do passageiro

Como você poderia dar um "chute" inicial para avaliar se um novo passageiro seria ou não sobrevivente no Titanic?

Podemos usar uma probabilidade conhecida como **a priori** que não leva em consideração qualquer característica do passageiro. Essa probabilidade é dada por:

$$\qquad P(sobrevivente)\qquad \mbox{e} \qquad P(não\hspace{1mm}sobrevivente).$$

Assuma $S$ como evento ser sobrevivente e $S^c$ como evento não ser sobrevivente.

Agora, calcule essas probabilidades a partir da base de dados. Lembra que você aqui pode usar o comando `value_counts()`.

In [ ]:
# Tabela de frequências relativas => Probabilidades marginais
dados.Survived.value_counts(True, sort=False).round(4)

Vamos guardar essas probabilidades em variáveis:

In [ ]:
priori = dados.Survived.value_counts(True, sort=False)
P_Sc = priori[0]
P_S = priori[1]

print(f'Probabilidade a priori de não sobreviver {P_Sc:0.5g}')
print(f'Probabilidade a priori de sobreviver {P_S:0.5g}')

E aí? Se chegasse alguém agora para ser um passageiro do Titanic, esse seria classificado como sobrevivente ou não sobrevivente?

E se chegassem 1.000 passageiros?

<div id="naivebayes"></div>

___
## Classificador Naive-Bayes

O Classificador Naive-Bayes permite que seja feito uma classificação, aqui, como sobrevivente ou não sobrevivente, porém levando em considerando **alguma(s) característica(s)** do passageiro. 

Vale ressaltar a importância de utilizar características (variáveis ou *features*) do passageiro que sejam associadas (dependentes) da variável *target* `Survived` que fornece a classificação.

Vamos entender o **Teorema de Bayes** utilizando a característica do passageiro descrita na variável `Sex`. Ainda, vamos utilizar os eventos $S$ para sobrevivente, $F$ para feminino e $M$ para masculino.

A probabilidade de **ser sobrevivente** dado cada categoria da variável `Sex` pode ser escrita como:

$$P(S|F)=\frac{P(S \cap F)}{P(F)}=\frac{P(F|S)P(S)}{P(F)}\hspace{3cm}\mbox{(I)}$$

e

$$P(S|M)=\frac{P(S \cap M)}{P(M)}=\frac{P(M|S)P(S)}{P(M)}\hspace{2.5cm}\mbox{(II)}$$


Por consequência, a probabilidade de **não ser sobrevivente** dado cada categoria da variável `Sex` pode ser escrita como:

$$P(S^c|F)=1-P(S|F)$$

e

$$P(S^c|M)=1-P(S|M)$$


Agora, vamos obter todas as probabilidade envolvidas do lado direito das equações $\mbox{(I)}$ e $\mbox{(II)}$ acima descritas.

Para facilitar, vamos obter tais probabilidades nos seguintes passos:

 * Calcular as probabilides $P(S)$, $P(S^c)$, $P(F)$ e $P(M)$. Lembra que você aqui pode usar o comando `value_counts()`.
 * Construir um conjunto de dados apenas com os sobreviventes e um outro apenas com os não sobreviventes.
 * Em cada conjunto de dados, calcular a probabilidade de ser cada categoria da variável `Sex`, ou seja, 
     * Entre os sobreviventes: calcular $𝑃(F|S)$ e $𝑃(M|S)$ a partir da base de dados. Aqui pode usar o comando `value_counts()`.
     * Entre os não sobreviventes: calcular $𝑃(F|S^c)$ e $𝑃(M|S^c)$ a partir da base de dados. Aqui pode usar o comando `value_counts()`.
          
  * Com todas essas probabilidades, calcular $P(S|F)$, $P(S^c|F)$, $P(S|M)$ e $P(S^c|M)$.

Vamos lá! **Passo-a-passo**:

* Calcule as probabilides $P(S)$, $P(S^c)$, $P(F)$ e $P(M)$. Lembra que você aqui pode usar o comando `value_counts()`.

In [ ]:
prob_surv = dados.Survived.value_counts(True, sort=False)
P_Sc = prob_surv[0]
P_S = prob_surv[1]

print(f'Probabilidade a priori de não sobreviver: {P_Sc:0.5g}')
print(f'Probabilidade a priori de sobreviver: {P_S:0.5g}')

In [ ]:
dados.Sex.value_counts(True, sort=False)

In [ ]:
prob_sex = dados.Sex.value_counts(True, sort=False)
P_F = prob_sex[0]
P_M = prob_sex[1]

print(f'Probabilidade de ser do sexo feminino: {P_F:0.5g}')
print(f'Probabilidade de ser do sexo masculino: {P_M:0.5g}')

 * Construa um conjunto de dados apenas com os sobreviventes e um outro apenas com os não sobreviventes.
 

In [ ]:
dados_S = dados[dados.Survived == 'Yes'].copy()
dados_S.head(5)

In [ ]:
dados_Sc = dados[dados.Survived == 'No'].copy()
dados_Sc.head(5)

* Entre os sobreviventes: calcule $𝑃(F|S)$ e $𝑃(M|S)$ a partir da base de dados. Aqui pode usar o comando `value_counts()`.
     

In [ ]:
dados_S.Sex.value_counts(True, sort=False)

In [ ]:
prob_sex_dado_S = dados_S.Sex.value_counts(True, sort=False)
P_F_dado_S = prob_sex_dado_S[0]
P_M_dado_S = prob_sex_dado_S[1]

print(f'Probabilidade de ser do sexo feminino entre os sobreviventes: {P_F_dado_S:0.5g}')
print(f'Probabilidade de ser do sexo masculino entre os sobreviventes: {P_M_dado_S:0.5g}')

* Entre os não sobreviventes: calcule $𝑃(F|S^c)$ e $𝑃(M|S^c)$ a partir da base de dados. Aqui pode usar o comando `value_counts()`.
          
  

In [ ]:
dados_Sc.Sex.value_counts(True, sort=False)

In [ ]:
prob_sex_dado_Sc = dados_Sc.Sex.value_counts(True, sort=False)
P_F_dado_Sc = prob_sex_dado_Sc[0]
P_M_dado_Sc = prob_sex_dado_Sc[1]

print(f'Probabilidade de ser do sexo feminino entre os sobreviventes: {P_F_dado_Sc:0.5g}')
print(f'Probabilidade de ser do sexo masculino entre os sobreviventes: {P_M_dado_Sc:0.5g}')

* Com todas essas probabilidades, calcule $P(S|F)$, $P(S^c|F)$, $P(S|M)$ e $P(S^c|M)$.

In [ ]:
P_S_dado_F = (P_F_dado_S * P_S)/P_F
P_Sc_dado_F = 1 - P_S_dado_F

print(f'Probabilidade de ser sobreviventdo se feminino: {P_S_dado_F:0.5g}')
print(f'Probabilidade de ser não sobrevivente se feminino: {P_Sc_dado_F:0.5g}')

In [ ]:
P_S_dado_M = (P_M_dado_S * P_S)/P_M
P_Sc_dado_M = 1 - P_S_dado_M

print(f'Probabilidade de ser sobreviventdo se masculino: {P_S_dado_M:0.5g}')
print(f'Probabilidade de ser não sobrevivente se masculino: {P_Sc_dado_M:0.5g}')

<div id="class"></div>

___
## Classificando os passageiros 

Considerando seu **Classificador Naive-Bayes**, classifique cada passageiro do *DataFrame* `dados`como `No` ou `Yes` considerando apenas a variável `Sex`.

No dataframe `dados`, guarde essas classificações em uma nova variável chamada `ClassBySex`. 

In [ ]:
# Classificando as mulheres
if (P_S_dado_F > P_Sc_dado_F):
    dados.loc[(dados.Sex=='female'),'ClassBySex'] = 'Yes'
else:
    dados.loc[(dados.Sex=='female'),'ClassBySex'] = 'No'
    

In [ ]:
# Classificando os homens
if (P_S_dado_M > P_Sc_dado_M):
    dados.loc[(dados.Sex=='male'),'ClassBySex'] = 'Yes'
else:
    dados.loc[(dados.Sex=='male'),'ClassBySex'] = 'No'

In [ ]:
dados.head(3)

<div id="sentimento"></div>

___
## Análise de sentimento

Faça uma análise de sentimento cruzando a variável `ClassBySex` com a real classificação `Survived` utilizando a função `pd.crosstab`.

In [ ]:
# Coloque seu código aqui

In [26]:
f'{(tab1.iloc[0,0]+tab1.iloc[1,1])*100:.5g}%'

'78.676%'

<div id="casa"></div>

## Para casa: Estudo Complementar

Vários pontos importantes podemos discutir com o classificador feito acima. Alguns seriam:

   * Poderia utilizar outra característica do passageiro para gerar a classificação acima como: `Pclass`, `Age`, entre outras? E ainda elas poderiam estar todas combinadas?
   * Foi avaliada a qualidade de classificação (Análise de Sentimento) considerando a mesma base de dados que criou o classificador. Como saber se um novo passageiro será classificado com a mesma qualidade medida na Análise de Sentimento anterior?

ESCREVA SUA RESPOSTA AQUI

___

Ainda, faça as tarefas a seguir:


<div id="classPclass"></div>

### Classificador Naive-Bayes com a variável `Pclass`

Refaça todas as etapas acima feita com a variável `Sex`, mas agora considerando a variável `Pclass`. Essa outra característica do passageiro classifica melhor se passageiro será ou não sobrevivente?


In [ ]:
# Coloque seu código aqui

<div id="testtrain"></div>

### Separando conjunto de dados em Treinamento (train)  e Teste (test)

Para aplicar a teoria de Machine Learning (ML), é necessário separar a base de dados em duas partes: treinamento e teste.

 * **Treinamento**: usualmente contém 75% das linhas (aqui, considere 670 passageiros) para treinar o modelo. Ou seja, qual perfil do passageiro tende a ser sobrevivente e qual perfil, não sobreviveu.
 * **Teste**: as demais linhas (aqui, 221 passageiros). Esses passageiros deverão ser classificacos como sobrevivente ou não sobrevivente segundo modelo treinado. Essa classificação via modelo deve ser confrontada com real classificação fazendo uma Análise de Sentimento, como é chamado pela área de ML.
 
Faça a separação de 670 passageiros para ser da base treinamento e os demais, como teste. Faça essa separação de forma aleatória.

O comando `shuffle` da biblioteca `sklearn.utils` pode ser uma alternativa, por ora, para realizar essa separação.

In [ ]:
from sklearn.utils import shuffle

In [ ]:
dados2 = dados.copy()
shuffle(dados)

In [ ]:
train = dados2.iloc[0:670,]
train.shape

In [ ]:
train.head()

In [ ]:
test = dados2.iloc[670:,]
test.shape

In [ ]:
test.head(5)

A partir daqui, treine seu classificador com o *DataFrame* `train` e avalie a qualidade do seu classificador com o *DataFrame* `test`.

In [ ]:
# Coloque seu código aqui